In [1]:
!pip install --upgrade scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 66.9 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.22.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.10.0, but you have google-cloud-bigquery 2.34.4 which is incompatible.
bigframes 0.22.0 requires google-cloud-storage>=2.0.0, but you have google-cloud-storage 1.44.0 which is incompatible.
bigframes 0.22.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.2.3 which is incompatible.
cesium 0.12.3 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
dataproc-jupyter-plugin 0.1.79 requires pydantic~=1.10.0, but you have pydantic 2.9.2 which is incompatible.


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import optuna
from catboost import CatBoostClassifier as CBC
import lightgbm as LGB
from sklearn.ensemble import HistGradientBoostingClassifier as HGBC
from sklearn.preprocessing import OrdinalEncoder as OE
from sklearn.model_selection import KFold, StratifiedKFold, StratifiedGroupKFold
from sklearn.metrics import accuracy_score

In [3]:
train = pd.read_csv('/kaggle/input/playground-series-s4e11/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s4e11/test.csv')
train.shape, test.shape

((140700, 20), (93800, 19))

In [4]:
train.head()

,id,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,0,Aaradhya,Female,49.0,Ludhiana,Working Professional,Chef,NaN,5.0,NaN,NaN,2.0,More than 8 hours,Healthy,BHM,No,1.0,2.0,No,0
1,1,Vivan,Male,26.0,Varanasi,Working Professional,Teacher,NaN,4.0,NaN,NaN,3.0,Less than 5 hours,Unhealthy,LLB,Yes,7.0,3.0,No,1
2,2,Yuvraj,Male,33.0,Visakhapatnam,Student,NaN,5.0,NaN,8.97,2.0,NaN,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
3,3,Yuvraj,Male,22.0,Mumbai,Working Professional,Teacher,NaN,5.0,NaN,NaN,1.0,Less than 5 hours,Moderate,BBA,Yes,10.0,1.0,Yes,1
4,4,Rhea,Female,30.0,Kanpur,Working Professional,Business Analyst,NaN,1.0,NaN,NaN,1.0,5-6 hours,Unhealthy,BBA,Yes,9.0,4.0,Yes,0


In [5]:
test.head()

,id,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness
0,140700,Shivam,Male,53.0,Visakhapatnam,Working Professional,Judge,NaN,2.0,NaN,NaN,5.0,Less than 5 hours,Moderate,LLB,No,9.0,3.0,Yes
1,140701,Sanya,Female,58.0,Kolkata,Working Professional,Educational Consultant,NaN,2.0,NaN,NaN,4.0,Less than 5 hours,Moderate,B.Ed,No,6.0,4.0,No
2,140702,Yash,Male,53.0,Jaipur,Working Professional,Teacher,NaN,4.0,NaN,NaN,1.0,7-8 hours,Moderate,B.Arch,Yes,12.0,4.0,No
3,140703,Nalini,Female,23.0,Rajkot,Student,NaN,5.0,NaN,6.84,1.0,NaN,More than 8 hours,Moderate,BSc,Yes,10.0,4.0,No
4,140704,Shaurya,Male,47.0,Kalyan,Working Professional,Teacher,NaN,5.0,NaN,NaN,5.0,7-8 hours,Moderate,BCA,Yes,3.0,4.0,No


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140700 entries, 0 to 140699
Data columns (total 20 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   id                                     140700 non-null  int64  
 1   Name                                   140700 non-null  object 
 2   Gender                                 140700 non-null  object 
 3   Age                                    140700 non-null  float64
 4   City                                   140700 non-null  object 
 5   Working Professional or Student        140700 non-null  object 
 6   Profession                             104070 non-null  object 
 7   Academic Pressure                      27897 non-null   float64
 8   Work Pressure                          112782 non-null  float64
 9   CGPA                                   27898 non-null   float64
 10  Study Satisfaction                     27897 non-null   

In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93800 entries, 0 to 93799
Data columns (total 19 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   id                                     93800 non-null  int64  
 1   Name                                   93800 non-null  object 
 2   Gender                                 93800 non-null  object 
 3   Age                                    93800 non-null  float64
 4   City                                   93800 non-null  object 
 5   Working Professional or Student        93800 non-null  object 
 6   Profession                             69168 non-null  object 
 7   Academic Pressure                      18767 non-null  float64
 8   Work Pressure                          75022 non-null  float64
 9   CGPA                                   18766 non-null  float64
 10  Study Satisfaction                     18767 non-null  float64
 11  Jo

In [8]:
trc = list(train.columns)
tec = list(test.columns)
NUM_COLS, CAT_COLS = [],[]
for i in trc:
    if i not in tec:
        TARGET = i
    elif train[i].dtype in [int,float]:
        NUM_COLS.append(i)
    else:
        CAT_COLS.append(i)

print((len(CAT_COLS) + len(NUM_COLS) + 1) == len(trc))
NUM_COLS.remove('id')

True


In [9]:
print(train['Name'].nunique())
train['Name'] = train['Name'].str.rstrip(' ')
train['Name'] = train['Name'].str.lstrip(' ')
print(train['Name'].nunique())

422
422


In [10]:
nns = train.Name.value_counts()
nns = nns.loc[nns<2]
nns.index

Index(['Rupika', 'Aaransh', 'Ijra', 'Aikash', 'Anishi', 'Aaranya', 'Anar',
       'Parha', 'Srinagar', 'Vidha',
       ...
       'Aariket', 'Aarat', 'Vohi', 'Vavya', 'Ishaam', 'Anhil', 'Rieta',
       'Zahra', 'Jathesh', 'Aarun'],
      dtype='object', name='Name', length=178)

In [11]:
train.loc[train.index[train.Name.isin(nns.index)],'Name'] ='rare'
train.Name.nunique()

245

In [12]:
train.Name.nunique()

245

In [13]:
oe = OE( handle_unknown = 'use_encoded_value', unknown_value= -1, encoded_missing_value= -2)
oe.fit(train[CAT_COLS])
train[CAT_COLS] = oe.transform(train[CAT_COLS])
test[CAT_COLS] = oe.transform(test[CAT_COLS])

In [14]:
for i in CAT_COLS:
    train[i] = train[i].astype(np.int64) + 2   
    test[i] = test[i].astype(np.int64) + 2
    print(f"{i}: ",train[i].isna().sum(),test[i].isna().sum())

Name:  0 0
Gender:  0 0
City:  0 0
Working Professional or Student:  0 0
Profession:  0 0
Sleep Duration:  0 0
Dietary Habits:  0 0
Degree:  0 0
Have you ever had suicidal thoughts ?:  0 0
Family History of Mental Illness:  0 0


In [15]:
# for i in NUM_COLS:
#     if train[i].isna().sum() >0 :
#         train[i] = train[i].fillna(np.mean(train[i]))
#         test[i] = test[i].fillna(np.mean(test[i]))                           
#         print(f"{i}: ",train[i].isna().sum(),test[i].isna().sum())

In [16]:
train.info()
#great, encoding fixed all by 6 columns of nulls

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140700 entries, 0 to 140699
Data columns (total 20 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   id                                     140700 non-null  int64  
 1   Name                                   140700 non-null  int64  
 2   Gender                                 140700 non-null  int64  
 3   Age                                    140700 non-null  float64
 4   City                                   140700 non-null  int64  
 5   Working Professional or Student        140700 non-null  int64  
 6   Profession                             140700 non-null  int64  
 7   Academic Pressure                      27897 non-null   float64
 8   Work Pressure                          112782 non-null  float64
 9   CGPA                                   27898 non-null   float64
 10  Study Satisfaction                     27897 non-null   

In [17]:
test.info() #even better, only 5 test columns now have nulls

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93800 entries, 0 to 93799
Data columns (total 19 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   id                                     93800 non-null  int64  
 1   Name                                   93800 non-null  int64  
 2   Gender                                 93800 non-null  int64  
 3   Age                                    93800 non-null  float64
 4   City                                   93800 non-null  int64  
 5   Working Professional or Student        93800 non-null  int64  
 6   Profession                             93800 non-null  int64  
 7   Academic Pressure                      18767 non-null  float64
 8   Work Pressure                          75022 non-null  float64
 9   CGPA                                   18766 non-null  float64
 10  Study Satisfaction                     18767 non-null  float64
 11  Jo

In [18]:
nns = train.Name.value_counts()
len(nns.loc[nns<2]), train.Name.nunique() ,train.Name.nunique()  - len(nns.loc[nns<2])

(0, 245, 245)

In [19]:
BOTH = CAT_COLS + NUM_COLS

sts = train[TARGET].value_counts()
sts[0] / sts.sum(), sts[1]/sts.sum()
wts = {0:0.8182871357498224, 1:0.18171286425017769}
wts

{0: 0.8182871357498224, 1: 0.18171286425017769}

In [20]:
trainsmall = train.sample(51000)
trainsmall.reset_index(inplace=True,drop=True)
trainsmall.head()

,id,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,99415,145,2,27.0,88,2,0,2.0,NaN,6.78,5.0,NaN,17,17,32,3,9.0,5.0,3,1
1,20918,95,3,24.0,19,3,12,NaN,1.0,NaN,NaN,5.0,31,9,35,3,11.0,2.0,2,0
2,122313,190,3,56.0,78,3,35,NaN,1.0,NaN,NaN,2.0,21,17,65,2,7.0,1.0,3,0
3,68097,233,3,45.0,15,3,13,NaN,4.0,NaN,NaN,1.0,29,9,78,3,3.0,5.0,3,0
4,15457,231,2,27.0,35,2,0,5.0,NaN,7.10,2.0,NaN,17,22,35,3,2.0,2.0,2,1


In [21]:
oo=0

def optim(trial):
    hg_params = {
        'loss': 'log_loss',
        'max_iter': 1512, 
        'learning_rate': trial.suggest_float('learning_rate',0.08,0.25),
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes',20,70),
        'max_depth': trial.suggest_int('max_depth',7,12),
        'min_samples_leaf':20,
        'l2_regularization': trial.suggest_int('l2_regularization',0,30),
        'categorical_features': CAT_COLS,
        'early_stopping': 'auto',
        "validation_fraction": 0.2, 
        'verbose':0,
        'class_weight': wts
    }
    FOLDS = 3
    scores = np.zeros(FOLDS)
    kf = KFold(n_splits = FOLDS)
    for i, (trndex,valdex) in enumerate(kf.split(trainsmall.id)):
        xtrain,ytrain = trainsmall.loc[trndex,BOTH], trainsmall.loc[trndex,TARGET]
        xval, yval    = trainsmall.loc[valdex,BOTH], trainsmall.loc[valdex,TARGET]
        
        model = HGBC(**hg_params) 
        model = model.fit(xtrain,ytrain)
        ypred = np.array(model.predict(xval))
        # ypred[ypred>=.5]=1
        # ypred[ypred<.5]=0
        ascore = accuracy_score(yval,ypred)
        scores[i] = ascore
    
    global oo
    print(f"{oo}: {np.mean(scores)}") 
    oo +=1
    return np.mean(scores)
study = optuna.create_study(direction='maximize')
study.optimize(optim,n_trials = 200)

#report the results 
print('Best parameters:', study.best_trial.params)
print('Best score:', study.best_value)

[I 2024-11-05 06:44:35,541] A new study created in memory with name: no-name-5a7ae698-6a56-47c6-af3f-8d787a8cea40
[I 2024-11-05 06:44:36,728] Trial 0 finished with value: 0.9232941176470589 and parameters: {'learning_rate': 0.23417151544985698, 'max_leaf_nodes': 21, 'max_depth': 7, 'l2_regularization': 2}. Best is trial 0 with value: 0.9232941176470589.


0: 0.9232941176470589


[I 2024-11-05 06:44:38,189] Trial 1 finished with value: 0.9238627450980391 and parameters: {'learning_rate': 0.20118056937676398, 'max_leaf_nodes': 67, 'max_depth': 7, 'l2_regularization': 0}. Best is trial 1 with value: 0.9238627450980391.


1: 0.9238627450980391


[I 2024-11-05 06:44:39,701] Trial 2 finished with value: 0.9259019607843136 and parameters: {'learning_rate': 0.24260571493621247, 'max_leaf_nodes': 43, 'max_depth': 10, 'l2_regularization': 10}. Best is trial 2 with value: 0.9259019607843136.


...

197: 0.9248235294117647


[I 2024-11-05 06:51:07,219] Trial 198 finished with value: 0.9255294117647059 and parameters: {'learning_rate': 0.24284687671025812, 'max_leaf_nodes': 63, 'max_depth': 12, 'l2_regularization': 8}. Best is trial 123 with value: 0.9278823529411765.


198: 0.9255294117647059


[I 2024-11-05 06:51:08,935] Trial 199 finished with value: 0.9248235294117646 and parameters: {'learning_rate': 0.24691860132751864, 'max_leaf_nodes': 64, 'max_depth': 12, 'l2_regularization': 9}. Best is trial 123 with value: 0.9278823529411765.


199: 0.9248235294117646
Best parameters: {'learning_rate': 0.24348917137475487, 'max_leaf_nodes': 68, 'max_depth': 11, 'l2_regularization': 10}
Best score: 0.9278823529411765
